Let’s create a table and perform database operations using direct SQL.

- Create table - courses
    - course_id - sequence generated integer and primary key
    - course_name - which holds alpha numeric or string values up to 60 characters
    - course_author - which holds the name of the author up to 40 characters
    - course_status - which holds one of these values (published, draft, inactive). Make sure to enforce to store one of these 3 values as part of course_status.
    - course_published_dt - which holds date type value.


- Insert data into courses using the data provided. Make sure id is system generated.

- Update the status of all the draft courses related to Python and Scala to published along with the course_published_dt using system date.
- Delete all the courses which are neither in draft mode nor published.
- Get count of all published courses by author and make sure output is sorted in descending order by count.

Create user 'itversity_courses_user'

```bash
psql -v ON_ERROR_STOP=1 --username "$POSTGRES_USER" --dbname "$POSTGRES_DB" <<EOSQL
    CREATE DATABASE itversity_courses_db;
    CREATE USER itversity_courses_user WITH ENCRYPTED PASSWORD 'courses_password';
    GRANT ALL ON DATABASE itversity_courses_db TO itversity_courses_user;
EOSQL
```

In [62]:
import psycopg2
from datetime import date

try:
    connection = psycopg2.connect(
        user="itversity_courses_user",
        password="courses_password",
        host="localhost",
        port="5432",
        database="itversity_courses_db"
    )
    cursor = connection.cursor()
    cursor.execute("""
        CREATE TYPE status AS ENUM ('published', 'draft', 'inactive');
        CREATE TABLE IF NOT EXISTS courses (
            course_id SERIAL PRIMARY KEY,
            course_name VARCHAR(60) NOT NULL,
            course_author VARCHAR(40) NOT NULL,
            course_status status,
            course_published_dt DATE
        );
    """)
    tuples = [
        ("Programming using Python", "Bob Dillon", "published", "2020-09-30"),
        ("Data Engineering using Python", "Bob Dillon", "published", "2020-07-15"),
        ("Data Engineering using Scala", "Elvis Presley", "draft", None),
        ("Programming using Scala", "Elvis Presley", "published", "2020-05-12"),
        ("Programming using Java", "Mike Jack", "inactive", "2020-08-10"),
        ("Web Applications - Python Flask", "Bob Dillon", "inactive", "2020-07-20"),
        ("Web Applications - Java Spring", "Mike Jack", "draft", None),
        ("Pipeline Orchestration - Python", "Bob Dillon", "draft", None),
        ("Streaming Pipelines - Python", "Bob Dillon", "published", "2020-10-05"),
        ("Web Applications - Scala Play", "Elvis Presley", "inactive", "2020-09-30"),
        ("Web Applications - Python Django", "Bob Dillon", "published", "2020-06-23"),
        ("Server Automation - Ansible", "Uncle Sam", "published", "2020-07-05")
    ]
    for tuple in range(len(tuples)):
        cursor.execute("""
            INSERT INTO courses (
                course_name,
                course_author,
                course_status,
                course_published_dt
            ) VALUES (%s, %s, %s, %s);
        """, tuples[tuple])
    cursor.execute("SELECT * FROM courses;")
    result_set = cursor.fetchall()
    for row in result_set:
        print(f"{row[0]} {row[1]} {row[2]} {row[3]} {row[4]}")
    print("\n")
    
    today = date.today()
    published_dt = today.strftime("%Y-%m-%d")
    cursor.execute(f"""
        UPDATE courses 
            SET course_status = 'published',
                course_published_dt = '{published_dt}' 
        WHERE (course_name like '%Scala%' OR course_name like '%Python%')
            AND (course_status = 'draft');
    """)
    cursor.execute("SELECT * FROM courses;")
    result_set = cursor.fetchall()
    for row in result_set:
        print(f"{row[0]} {row[1]} {row[2]} {row[3]} {row[4]}")
    print("\n")

    cursor.execute("""
        DELETE FROM courses
        WHERE course_status NOT IN ('draft','published');
    """)
    cursor.execute("SELECT * FROM courses;")
    result_set = cursor.fetchall()
    for row in result_set:
        print(f"{row[0]} {row[1]} {row[2]} {row[3]} {row[4]}")
    print("\n")

    cursor.execute("""
        SELECT
            course_author,
            count(course_author) AS course_count
        FROM courses
        GROUP BY course_author
        ORDER BY 1,2 DESC;
    """)
    result_set = cursor.fetchall()
    for row in result_set:
        print(f"{row[0]} {row[1]}")
    print("\n")

    connection.commit()
except Exception as sql_e:
    print(sql_e)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print('PostgreSQL connection closed')

1 Programming using Python Bob Dillon published 2020-09-30
2 Data Engineering using Python Bob Dillon published 2020-07-15
3 Data Engineering using Scala Elvis Presley draft None
4 Programming using Scala Elvis Presley published 2020-05-12
5 Programming using Java Mike Jack inactive 2020-08-10
6 Web Applications - Python Flask Bob Dillon inactive 2020-07-20
7 Web Applications - Java Spring Mike Jack draft None
8 Pipeline Orchestration - Python Bob Dillon draft None
9 Streaming Pipelines - Python Bob Dillon published 2020-10-05
10 Web Applications - Scala Play Elvis Presley inactive 2020-09-30
11 Web Applications - Python Django Bob Dillon published 2020-06-23
12 Server Automation - Ansible Uncle Sam published 2020-07-05


1 Programming using Python Bob Dillon published 2020-09-30
2 Data Engineering using Python Bob Dillon published 2020-07-15
4 Programming using Scala Elvis Presley published 2020-05-12
5 Programming using Java Mike Jack inactive 2020-08-10
6 Web Applications - Python F